In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = None
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = r"C:\Users\Kraveta\Desktop\stanford\assignment1\cs231n\datasets\cifar-10-batches-py"

    
    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside `cs231n/classifiers/softmax.py`.


In [ ]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

In [30]:
y_onehot = np.zeros((y_dev.size, y_dev.max()+1))
y_onehot[np.arange(y_dev.size),y_dev] = 1


In [31]:
y_onehot

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
np.oney_dev

array([7, 3, 8, 7, 7, 3, 6, 2, 9, 3, 7, 0, 0, 9, 3, 8, 9, 5, 8, 8, 3, 7,
       9, 1, 2, 9, 8, 2, 8, 5, 8, 7, 3, 8, 8, 6, 7, 5, 2, 7, 8, 6, 6, 1,
       3, 3, 9, 5, 5, 3, 0, 6, 2, 8, 4, 5, 2, 2, 1, 1, 8, 9, 8, 8, 9, 3,
       0, 4, 7, 5, 2, 4, 0, 2, 9, 5, 8, 8, 2, 6, 1, 3, 7, 4, 1, 1, 2, 0,
       6, 8, 9, 9, 7, 8, 5, 9, 7, 5, 2, 7, 8, 6, 7, 6, 6, 7, 4, 0, 5, 0,
       0, 6, 1, 7, 3, 9, 9, 6, 3, 8, 9, 0, 9, 9, 3, 5, 3, 7, 4, 3, 1, 7,
       3, 8, 2, 9, 7, 3, 1, 0, 1, 0, 7, 5, 7, 5, 2, 5, 8, 3, 5, 7, 0, 2,
       9, 7, 2, 1, 3, 6, 9, 1, 9, 6, 7, 2, 7, 5, 6, 0, 7, 1, 8, 3, 0, 1,
       5, 7, 1, 7, 4, 8, 1, 1, 7, 5, 7, 4, 2, 0, 2, 3, 8, 3, 0, 6, 0, 6,
       7, 5, 1, 5, 1, 6, 1, 1, 8, 1, 8, 7, 9, 5, 5, 8, 6, 7, 9, 9, 5, 0,
       7, 1, 1, 1, 5, 8, 8, 7, 9, 0, 0, 2, 2, 3, 1, 8, 8, 1, 7, 9, 8, 9,
       8, 0, 0, 3, 4, 9, 6, 8, 2, 6, 9, 5, 8, 7, 3, 4, 5, 4, 5, 4, 6, 4,
       0, 7, 0, 6, 9, 6, 3, 7, 9, 4, 2, 7, 4, 2, 8, 7, 2, 6, 5, 3, 5, 6,
       5, 7, 9, 2, 0, 9, 2, 8, 2, 2, 0, 5, 6, 6, 2,

In [40]:
X_dev - X_dev.max(1).reshape(-1,1)

array([[-157.1567551 , -110.49659184,  -18.98877551, ..., -107.37681633,
        -147.91442857, -130.51485714],
       [-143.11289796, -132.45273469, -110.94491837, ..., -171.33295918,
        -160.87057143,  -83.471     ],
       [-163.78295918, -206.12279592, -221.61497959, ..., -103.00302041,
        -134.54063265, -143.14106122],
       ...,
       [-218.22755102, -202.56738776, -184.05957143, ..., -230.44761224,
        -214.98522449, -126.58565306],
       [-121.13338776, -124.47322449, -142.96540816, ...,  -81.35344898,
        -108.89106122,  -86.4914898 ],
       [ -39.34763265,  -44.68746939,  -41.17965306, ...,  -37.56769388,
         -26.10530612, -153.70573469]])

In [51]:
(np.exp(X_dev.dot(W))/np.exp(X_dev.dot(W)).sum(axis=1).reshape(-1,1))[0]

array([0.06885124, 0.14396844, 0.07726356, 0.10081266, 0.08400108,
       0.07546771, 0.13448628, 0.09717027, 0.13284607, 0.08513268])

In [59]:
(np.exp(X_dev[0].dot(W))/np.exp(X_dev[0].dot(W)).sum().reshape(-1,1))

array([[0.06885124, 0.14396844, 0.07726356, 0.10081266, 0.08400108,
        0.07546771, 0.13448628, 0.09717027, 0.13284607, 0.08513268]])

In [113]:
f = X_dev.dot(W)[0]
f = f - f.max().reshape(-1,1)
(np.exp(f)/np.exp(f).sum(axis=1).reshape(-1,1))[0]

array([0.06885124, 0.14396844, 0.07726356, 0.10081266, 0.08400108,
       0.07546771, 0.13448628, 0.09717027, 0.13284607, 0.08513268])

In [70]:
W_toy = np.array([[0.01, -0.05, 0.1, 0.05],[0.7, 0.2, 0.05, 0.16],
                  [0, -0.45, -0.2, 0.03]]).transpose()
X_toy =  np.array([[-15, 22, -44, 56]])
Y_toy = np.array([2])
b = np.array([0,0.2,-0.3])

In [73]:
np.exp(X_toy.dot(W_toy[:,2]) + b[2]) / np.sum(np.exp(X_toy.dot(W_toy) + b))

array([0.35338733])

In [77]:
def softmax_loss_naive(W, X, y, reg):
  """
  Softmax loss function, naive implementation (with loops)
  Inputs have dimension D, there are C classes, and we operate on minibatches
  of N examples.
  Inputs:
  - W: A numpy array of shape (D, C) containing weights.
  - X: A numpy array of shape (N, D) containing a minibatch of data.
  - y: A numpy array of shape (N,) containing training labels; y[i] = c means
    that X[i] has label c, where 0 <= c < C.
  - reg: (float) regularization strength
  Returns a tuple of:
  - loss as single float
  - gradient with respect to weights W; an array of same shape as W
  """
  # Initialize the loss and gradient to zero.
  loss = 0.0
  dW = np.zeros_like(W)

  #############################################################################
  # TODO: Compute the softmax loss and its gradient using explicit loops.     #
  # Store the loss in loss and the gradient in dW. If you are not careful     #
  # here, it is easy to run into numeric instability. Don't forget the        #
  # regularization!                                                           #
  #############################################################################
  scores = X.dot(W)
  num_train = X.shape[0]
  num_classes = W.shape[1]

  # Softmax Loss
  for i in range(num_train):
    f = scores[i] - np.max(scores[i]) # avoid numerical instability
    softmax = np.exp(f)/np.sum(np.exp(f))
    loss += -np.log(softmax[y[i]])
    # Weight Gradients
    for j in range(num_classes):
      dW[:,j] += X[i] * softmax[j]
    dW[:,y[i]] -= X[i]

  # Average
  loss /= num_train
  dW /= num_train

  # Regularization
  loss += reg * np.sum(W * W)
  dW += reg * 2 * W 

  #############################################################################
  #                          END OF YOUR CODE                                 #
  #############################################################################

  return loss, dW

In [173]:
def softmax_loss_naive_mine(W, X, y, reg):
    """
    Softmax loss function, naive implementation (with loops)

    Inputs have dimension D, there are C classes, and we operate on minibatches
    of N examples.

    Inputs:
    - W: A numpy array of shape (D, C) containing weights.
    - X: A numpy array of shape (N, D) containing a minibatch of data.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c means
      that X[i] has label c, where 0 <= c < C.
    - reg: (float) regularization strength

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W; an array of same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    #############################################################################
    # TODO: Compute the softmax loss and its gradient using explicit loops.     #
    # Store the loss in loss and the gradient in dW. If you are not careful     #
    # here, it is easy to run into numeric instability. Don't forget the        #
    # regularization!                                                           #
    #############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    num_exm = X.shape[0]
    num_cls = W.shape[1]

    y_onehot = np.zeros((y_dev.size, y_dev.max()+1))
    y_onehot[np.arange(y_dev.size),y_dev] = 1
    
    for i in range(num_exm):
        f = X[i].dot(W)
        f = f - f.max().reshape(-1,1)
        scores = (np.exp(f)/np.exp(f).sum(axis=1).reshape(-1,1))[0]
        for j in range(num_cls):
            if y_onehot[i][j] == 1:
                loss += -np.log(scores[j]*y_onehot[i][j])
            dW[:,j] += X[i] * scores[j]
        dW[:,y[i]] -= X[i]
    
    loss /= num_exm
    dW /= num_exm
    
    loss += reg * np.sum(W * W)
    dW += reg * 2 * W 

    return loss, dW

In [175]:
grad2

array([[-2.37635250e-01, -1.52010931e+00,  1.21349485e+00, ...,
         1.36109107e+00, -3.67805058e+00, -4.13055184e+00],
       [-2.21296079e+00, -1.14779297e+00,  8.20819949e-01, ...,
         9.59633791e-01, -4.53322838e+00, -4.14384124e+00],
       [-3.71473805e+00, -1.24894523e+00,  1.88950112e+00, ...,
         1.32001305e+00, -6.56239182e+00, -5.42863783e+00],
       ...,
       [-6.43872778e-01, -1.44151633e+00,  8.05946372e-01, ...,
        -2.37611216e+00,  1.36080534e-02,  1.80746528e-01],
       [-1.83702218e+00, -2.52931950e+00,  1.17868206e+00, ...,
         2.39064332e-01, -2.25795862e+00, -3.94773109e-01],
       [-5.39611081e-03,  2.54372399e-03,  1.10774705e-02, ...,
        -2.34435929e-02, -2.07070699e-02,  4.84661666e-03]])

In [177]:
grad2

array([[-2.37635250e-01, -1.52010931e+00,  1.21349485e+00, ...,
         1.36109107e+00, -3.67805058e+00, -4.13055184e+00],
       [-2.21296079e+00, -1.14779297e+00,  8.20819949e-01, ...,
         9.59633791e-01, -4.53322838e+00, -4.14384124e+00],
       [-3.71473805e+00, -1.24894523e+00,  1.88950112e+00, ...,
         1.32001305e+00, -6.56239182e+00, -5.42863783e+00],
       ...,
       [-6.43872778e-01, -1.44151633e+00,  8.05946372e-01, ...,
        -2.37611216e+00,  1.36080534e-02,  1.80746528e-01],
       [-1.83702218e+00, -2.52931950e+00,  1.17868206e+00, ...,
         2.39064332e-01, -2.25795862e+00, -3.94773109e-01],
       [-5.39611081e-03,  2.54372399e-03,  1.10774705e-02, ...,
        -2.34435929e-02, -2.07070699e-02,  4.84661666e-03]])

In [169]:
loss1, grad1 = softmax_loss_naive(W, X_dev, y_dev, 0.0)
loss1

2.3923282639759136

In [174]:
loss2, grad2 = softmax_loss_naive_mine(W, X_dev, y_dev, 0.0)
loss2

2.3923282639759127

**Inline Question 1**

Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

$\color{blue}{\textit Your Answer:}$ *Fill this in* 



In [ ]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

In [ ]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.

from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################

# Provided as a reference. You may or may not want to change these hyperparameters
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question 2** - *True or False*

Suppose the overall training loss is defined as the sum of the per-datapoint loss over all training examples. It is possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

$\color{blue}{\textit Your Answer:}$


$\color{blue}{\textit Your Explanation:}$



In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])